## NFL Fantasy Season Simulator - 2023/2024
### Created by Gian Favero ([@faverogian](https://github.com/faverogian)) on August 9, 2024
An in-depth analysis of draft strategy for optimal fantasy football performance, using data-driven approaches and simulations.
This project is licensed under the MIT License.

In [758]:
# imports
import pandas as pd

### Loading the data

Here we are using a consensus ADP 2023 ranking from FantasyPros according to 3 different sources: ESPN, Yahoo, and Sleeper for a 12 team league with half PPR scoring.

In [759]:
# Load ADP data
ADP_23 = pd.read_csv('data\FantasyPros_2023_Overall_ADP_Rankings.csv')

# Change position names to remove numbers
ADP_23['POS'] = ADP_23['POS'].str.translate(str.maketrans('', '', '1234567890'))
print('Sample of ADP data:')
print(ADP_23.head(), '\n')

# Load Half PPR data
HALF_PPR_23 = pd.read_csv('data\FantasyPros_Fantasy_Football_Points_HALF.csv')
print('Sample of Half PPR data:')
print(HALF_PPR_23.head())

Sample of ADP data:
   Rank               Player Team Bye POS  Yahoo  Sleeper  RTSports  AVG
0     1     Justin Jefferson  MIN  13  WR    1.0      1.0       1.0  1.0
1     2  Christian McCaffrey   SF   9  RB    2.0      2.0       2.0  2.0
2     3        Ja'Marr Chase  CIN   7  WR    3.0      3.0       3.0  3.0
3     4        Austin Ekeler  WAS  14  RB    4.0      4.0       4.0  4.0
4     5          Tyreek Hill  MIA  10  WR    5.0      6.0       5.0  5.3 

Sample of Half PPR data:
     #               Player Pos Team     1     2     3     4     5     6  ...  \
0  1.0           Josh Allen  QB  BUF    12  23.7  22.3  36.5  28.8  14.9  ...   
1  2.0          Jalen Hurts  QB  PHI  12.5  26.2  21.9  24.2  28.3  22.9  ...   
2  3.0  Christian McCaffrey  RB   SF  24.4    21  20.4  45.2  12.8  12.7  ...   
3  4.0         Dak Prescott  QB  DAL   6.3  19.6  15.4  14.3   7.3  24.9  ...   
4  5.0        Lamar Jackson  QB  BAL   7.6  22.9  28.2  28.1  10.9  18.1  ...   

     11    12    13    14   

### Simulating the draft

We assume the number of draft rounds is equal to the number of starting roster spots, neglecting the bench. 

In [760]:
class DraftConfiguration:
    order_system = 'snake' # snake or linear
    number_of_teams = 12
    roster = {
        'QB': 1,
        'RB': 2,
        'WR': 2,
        'TE': 1,
        'FLEX': 1,
        'K': 1,
        'DST': 1,
        'BENCH': 6,
        'nan': 0
     }

We assume a team can employ a 'BPA', 'Robust RB', 'WR Heavy', 'Early TE', or 'Early QB' strategy. 

In [761]:
class DraftStrategy:
    BPA = 'BPA' # Best Player Available
    RB_HEAVY = 'RB Heavy' # Fill RB slots first, then WR, then BPA
    WR_HEAVY = 'WR Heavy' # Fill WR slots first, then RB, then BPA
    EARLY_QB = 'Early QB' # Draft QB in first 3 rounds
    EARLY_TE = 'Early TE' # Draft TE in first 3 rounds

From here we can set up our "league". To examine a particular draft strategy, we set our team to the one in question and all others to BPA.

In [762]:
import random
from nfl_fantasy.roster import FantasyRoster

config = DraftConfiguration()
strategy = DraftStrategy.EARLY_TE

# Generate team with a strategy under study
team = FantasyRoster(config.roster.copy(), strategy)

# Generate league
fantasy_league = [team] + [FantasyRoster(config.roster.copy(), DraftStrategy.BPA) for i in range(config.number_of_teams-1)]

# Generate draft order
if config.order_system == 'snake':
    forward_draft_order = list(range(config.number_of_teams))
    random.shuffle(forward_draft_order)
    draft_order = forward_draft_order + forward_draft_order[::-1]
    rounds = sum(config.roster.values()) // 2
    draft_order *= rounds
    draft_order += forward_draft_order
    
else:
    draft_order = list(range(config.number_of_teams))
    rounds = sum(config.roster.values())
    draft_order *= rounds

# Make version of the ADP data for the draft
ADP_23_draft = ADP_23.copy() 

# Start the draft
for i, pick in enumerate(draft_order):
    team = fantasy_league[pick]
    player = team.choose_player(ADP_23_draft)
    bpa = ADP_23_draft.iloc[0]
    ADP_23_draft = ADP_23_draft[ADP_23_draft['Player'] != player['Player']]

# Print the results
for i, team in enumerate(fantasy_league):
    print(f'Strategy: {team.strategy}')
    for player in team.players:
        print(f"    ADP: {player['AVG']}  {player['Player']} ({player['POS']})")
    break

Strategy: Early TE
    ADP: 2.0  Christian McCaffrey (RB)
    ADP: 23.3  Jaylen Waddle (WR)
    ADP: 29.7  Mark Andrews (TE)
    ADP: 47.3  Jonathan Taylor (RB)
    ADP: 50.3  George Kittle (TE)
    ADP: 71.3  Alvin Kamara (RB)
    ADP: 75.0  Christian Kirk (WR)
    ADP: 96.3  Brian Robinson Jr. (RB)
    ADP: 101.3  Jamaal Williams (RB)
    ADP: 121.7  Dalton Kincaid (TE)
    ADP: 124.7  JuJu Smith-Schuster (WR)
    ADP: 146.7  Tyler Higbee (TE)
    ADP: 151.3  Baltimore Ravens (DST)
    ADP: 170.7  Russell Wilson (QB)
    ADP: 204.0  Brandon McManus (K)


### Simulating the Season

By comparing total points for across the entire season, we can approximate how well such a team would have done. This obviously grossly underestimates the notion of trades, waiver pickups, and the use of a bench. However, it is an assumption that follows the claim that with modern information at our disposal such transactions provide parallel success at best. The optimal line-up is provided on a weekly basis - however this does not account for season ending injuries at positions with no substitute. Monte Carlo simulations should aid in fixing this issue (e.g., drafting Anthony Richardson in 2023).

In [763]:
def get_week_points(team: FantasyRoster, week: int, half_ppr: pd.DataFrame, roster: dict) -> float:
    # Get the players for the team
    players = [player['Player'] for player in team.players]
    
    # Get the points for the players for the week
    points = half_ppr[half_ppr['Player'].isin(players)][['Player', str(week), 'Pos']]

    # Replace 'BYE' and '-' with 0
    points = points.replace('BYE', 0)
    points = points.replace('-', 0)
    points[str(week)] = points[str(week)].astype(float)

    # Sort the players by points
    points = points.sort_values(by=[str(week)], ascending=False)

    # Get the optimal lineup for the week using the roster configuration
    roster = roster.copy()
    for index, row in points.iterrows():
        pos = row['Pos']
        if roster.get(pos, 0) > 0:
            roster[pos] -= 1
        elif roster.get('FLEX', 0) > 0 and (pos == 'RB' or pos == 'WR'):
            roster['FLEX'] -= 1
        else:
            # Pop from the dataframe
            points = points.drop(index)

    # Return the points for the week
    return points[str(week)].sum()

points = 0
for team in fantasy_league:
    print(f'Strategy: {team.strategy}')
    for weeks in range(1, 17):
        points += get_week_points(team, 11, HALF_PPR_23, config.roster)
    break
print(f'Total points: {points}')

Strategy: Early TE
Total points: 1579.2000000000005
